## Reconstruction of coal-ice melting  

Author: atekawade@anl.gov, Aniket Tekawade  
Data contributor: nikitinvv@anl.gov, Viktor Nikitin  

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import sys
from ct_segnet import viewer
# from features3D import FeatureExtractor4D
from tomo_encoders.patches import Patches
import tensorflow as tf
from tomo_encoders.neural_nets.porosity_encoders import build_CAE_3D
sys.path.append('../.')
from recon4D import DataGetter
import time
from features4D import FeatureExtraction4D

In [2]:
# Reconstruction stuff
fnames = ['/data02/MyArchive/coalice/melting_086.h5', \
          '/data02/MyArchive/coalice/flat_fields_melting_086.h5', \
          '/data02/MyArchive/coalice/dark_fields_melting_086.h5']
ntheta = 361 # these many projections per 180 degree spin
recon_params = {"mask_ratio" : None, \
                "contrast_s" : 0.01}

In [3]:
dget = DataGetter(*fnames, ntheta)
idx = 0
vol = dget.reconstruct_window(idx,dget.find_center(idx), **recon_params)
# fig, ax = plt.subplots(1,3, figsize = (14,6))
# viewer.view_midplanes(vol = vol, ax = ax)   

Shape of projection image: (1, 600, 960)
center = 471.00


## Train auto-encoder for embedding  

In [4]:
# Feature Extraction stuff
model_path = '/data02/MyArchive/aisteer_3Dencoders/models/coalice'

model_size = (64,64,64)
model_params = {"n_filters" : [16, 32, 64],\
                "n_blocks" : 3,\
                "activation" : 'lrelu',\
                "batch_norm" : True,\
                "hidden_units" : [128, 128],\
                "isconcat" : [True, True, True],\
                "pool_size" : [2,2,2],\
                "stdinput" : False}

# model_size = (64,64,64)
# model_params = {"n_filters" : [32, 64, 128],\
#                 "n_blocks" : 3,\
#                 "activation" : 'lrelu',\
#                 "batch_norm" : True,\
#                 "hidden_units" : [128, 32],\
#                 "isconcat" : [True, True, True],\
#                 "pool_size" : [2,2,2],\
#                 "stdinput" : False}

training_params = {"sampling_method" : "random-fixed-width", \
                   "batch_size" : 24, \
                   "n_epochs" : 5,\
                   "random_rotate" : True, \
                   "add_noise" : 0.15, \
                   "max_stride" : 1}
descriptor_tag = 'coal-ice-tzero-bin1'

In [5]:
fe = FeatureExtraction4D(vol.shape, model_initialization = 'define-new', \
                         model_size = model_size, \
                         descriptor_tag = descriptor_tag, \
                         **model_params)

In [6]:
# vol = fe._normalize_volume(vol)
# dg = fe.data_generator(vol, training_params["batch_size"], \
#                        training_params["sampling_method"], \
#                        max_stride = training_params["max_stride"], \
#                        random_rotate = training_params["random_rotate"], \
#                        add_noise = training_params["add_noise"])

In [7]:
fe.train_CAE_selfsupervised(vol, training_params["batch_size"], \
                            training_params["sampling_method"], \
                            training_params["n_epochs"],\
                            max_stride = training_params["max_stride"], \
                            random_rotate = training_params["random_rotate"], \
                            add_noise = training_params["add_noise"])

Epoch 1/5
16/16 [==============================] - 24s 995ms/step - loss: 0.0147
Epoch 2/5
16/16 [==============================] - 16s 1s/step - loss: 0.0049
Epoch 3/5
16/16 [==============================] - 16s 1s/step - loss: 0.0037
Epoch 4/5
16/16 [==============================] - 16s 1s/step - loss: 0.0032
Epoch 5/5
16/16 [==============================] - 16s 1s/step - loss: 0.0030
training time = 89.53 seconds


In [8]:
fe.save_models(model_path)